In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
import time
import mahotas as mh
from skimage import measure
from tqdm.notebook import tqdm
from image.controller import ImageController

/home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
data_dir = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/wild_type_raw/'
save_dir = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/04-14/wild_type_cs_1/binary_mask_9/'

In [5]:
LOGGING_IN_NOTEBOOK = True
log_path = save_dir + 'log.txt'
crop_path = save_dir + 'original_crop/'
binary_mask_path = save_dir + 'binary_mask/'
compare_plot_path = save_dir + 'compare_plots/'
compare_labeled_plot_path = save_dir + 'compare_labeled_plots/'

In [6]:
if not LOGGING_IN_NOTEBOOK:
    sys.stdout = open(log_path, 'w')

In [7]:
# NOTE: one single ommatidium has 7 rhabdomeres.
# Value NUM_RHABDOMERE and values related to threshold are not to be changed
NUM_RHABDOMERE = 7

THRESHOLD_BRACKET = 40
LEAST_POSSIBLE_THRESHOLD = 1
MOST_POSSIBLE_THRESHOLD = 255

# Change these values as per different datasets
# Values for Wild_Type_CS_1
#     REMOVE_SMALL_REGION = True
#     REGION_SIZE = 750
#     LEAST_PIXEL_COUNT = 20000
#     LEAST_OMMATIDIA_COUNT = 1
# Values for NINA_D1_MUTANT
#     CROP_SIZE = (512, 512)
#     REMOVE_SMALL_REGION = False
#     REGION_SIZE = 750
#     LEAST_PIXEL_COUNT = 20000
#     LEAST_OMMATIDIA_COUNT = 12
CROP_SIZE = (512, 512)
REMOVE_SMALL_REGION = True
REGION_SIZE = 750
LEAST_PIXEL_COUNT = 20000
LEAST_OMMATIDIA_COUNT = 1

In [8]:
is_mask_bad = True
good_mask_count, bad_mask_count = 0, 0
file_count = 0

for file in tqdm(sorted(os.listdir(data_dir))):
    start_time = time.time()
    
    if file.endswith(".tif"):
        
        num_rhabdomeres, num_ommatidia = 0, 0
        best_threshold, best_accuracy, threshold_value = 0, 0, 0
        
        file_count += 1
        print("Image Name: " + file)
        
        # Read file
        img_file = os.path.join(data_dir, file)
        data = ImageController.read(img_file)
        
        # Normalize and crop
        # crop size defaults to (512, 512)
        normalized_data = ImageController.normalize(data)
        crop_data = ImageController.center_crop(normalized_data, crop_size=CROP_SIZE)
        
        # Save original image cropped
        data_crop = ImageController.center_crop(data, crop_size=CROP_SIZE)
        ImageController.save(os.path.join(crop_path, file), data_crop)
        
        # Apply guassian filter
        # can pass sigma as argument, defaults to 3
        smoothed_data = ImageController.smooth(crop_data)
        
        # Playing around with threshold values.
        # Start with mean threshold
        # Checking accuracy for 40 threshold up and down the mean value
        # Has to be done due to contrast variation in data
        threshold_value = int(smoothed_data.mean())
        threshold_start = LEAST_POSSIBLE_THRESHOLD
        threshold_limit = MOST_POSSIBLE_THRESHOLD
        if (threshold_value - THRESHOLD_BRACKET > LEAST_POSSIBLE_THRESHOLD):
            threshold_start = int(threshold_value) - THRESHOLD_BRACKET
        if (threshold_value + THRESHOLD_BRACKET < MOST_POSSIBLE_THRESHOLD):
            threshold_limit = int(threshold_value) + THRESHOLD_BRACKET
        
        # Loop over all threshold values and get threshold with best accuracy
        for threshold in range(threshold_start, threshold_limit, 1):
            # Apply thresholding
            threshold_data = ImageController.threshold(smoothed_data, threshold_value=threshold)
            # region labelling
            labeled_data, nr_count = ImageController.label(threshold_data)
            # remove regions
            # whether to remove small or large regions should be decided on the basis of dataset
            if REMOVE_SMALL_REGION:
                selected_labeled_data = ImageController.remove_small_regions(labeled_data, region_size=REGION_SIZE)
            else :
                selected_labeled_data = ImageController.remove_large_regions(labeled_data, region_size=REGION_SIZE)
            # Get binary mask of region removed labeled data
            binary_mask_data = ImageController.binary_mask(selected_labeled_data)
            # Apply close operation on binary mask
            segmented_image = ImageController.close_binary_mask(binary_mask_data)
            # count pixels in mask
            num_pixels = int(np.sum(segmented_image))
            # check accuracy of mask
            test_accuracy, test_num_rhabdomeres = ImageController.check_accuracy(binary_mask_data)
            # get best accuracy and satisfy least pixel count value
            if test_accuracy > best_accuracy and num_pixels > LEAST_PIXEL_COUNT:
                best_accuracy = test_accuracy
                best_threshold = threshold
                num_rhabdomeres = test_num_rhabdomeres
           
        # Run pipeline one last time for best threshold value
        threshold_data = ImageController.threshold(smoothed_data, threshold_value=best_threshold)
        labeled_data, nr_count = ImageController.label(threshold_data)
        if REMOVE_SMALL_REGION:
            selected_labeled_data = ImageController.remove_small_regions(labeled_data, region_size=REGION_SIZE)
        else :
            selected_labeled_data = ImageController.remove_large_regions(labeled_data, region_size=REGION_SIZE)
        binary_mask_data = ImageController.binary_mask(selected_labeled_data)
        closed_binary_mask_data = ImageController.close_binary_mask(binary_mask_data)
        segmented_image = ImageController.binary_image(closed_binary_mask_data)
        
        # count number of pixels in segmentation mask
        num_pixels = np.sum(segmented_image)
        
        # Save binary_mask, compare_labeled_plot and conpare_plot for best accuracy threshold
        compare_labeled_path = os.path.join(compare_labeled_plot_path, file)
        compare_labeled_plot_name = compare_labeled_path.split('.')[0] + '.png'
        
        compare_path = os.path.join(compare_plot_path, file)
        compare_plot_name = compare_path.split('.')[0] + '.png'
        
        binary_mask_name = os.path.join(binary_mask_path, file)
        
        # Save compare_labeled_plots for all images
        ImageController.display_and_save_compare_labeled(data_crop, 
                                                         binary_mask_data, 
                                                         segmented_image, 
                                                         compare_labeled_plot_name,
                                                         show_in_notebook=False)
        
        print("Threshold mean: ", smoothed_data.mean())
        print("Threshold best: ", best_threshold)
        print("Accuracy of mask: ", best_accuracy)
        
        num_ommatidia = round(num_rhabdomeres / NUM_RHABDOMERE)
        # if number of ommatidia in image and number of pixels in image
        # satisfy least count, then segmentation mask is good.
        if num_pixels > LEAST_PIXEL_COUNT and num_ommatidia >= LEAST_OMMATIDIA_COUNT:
            good_mask_count += 1
            print("Good segmentation mask")
            # Save binary mask and compare plot only for good mask image
            ImageController.save_compare(data_crop, segmented_image, compare_plot_name)
            ImageController.save(binary_mask_name, segmented_image)
        else:
            bad_mask_count += 1
            print("Bad segmentation mask")
            
        print("Possible number of rhabdomeres: ", num_rhabdomeres)
        print("Total number of pixels in the mask: ", num_pixels)
        
        end_time = time.time()
        print("Total Time Taken: ", (end_time - start_time), " seconds")
        
        print("Number of good masks so far: ", good_mask_count)
        print("Number of bad masks so far: ", bad_mask_count)
        print("Total files processed so far: ", file_count)
        
        print("---------------------------------------------------------")

print("Number of good masks: ", good_mask_count)
print("Number of bad masks: ", bad_mask_count)
print("Total processed images:", file_count)

  0%|          | 0/301 [00:00<?, ?it/s]

Image Name: 000000_000000_000001_000000.tif
Threshold mean:  84.00180453869885
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  5.621718883514404  seconds
Number of good masks so far:  0
Number of bad masks so far:  1
Total files processed so far:  1
---------------------------------------------------------
Image Name: 000000_000000_000003_000000.tif
Threshold mean:  88.85461246224986
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  6.976192951202393  seconds
Number of good masks so far:  0
Number of bad masks so far:  2
Total files processed so far:  2
---------------------------------------------------------
Image Name: 000000_000000_000012_000000.tif
Threshold mean:  89.06426176483376
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomere

Threshold mean:  72.82209403523596
Threshold best:  87
Accuracy of mask:  0.42857142857142855
Bad segmentation mask
Possible number of rhabdomeres:  3
Total number of pixels in the mask:  28145
Total Time Taken:  5.569064140319824  seconds
Number of good masks so far:  0
Number of bad masks so far:  21
Total files processed so far:  21
---------------------------------------------------------
Image Name: 000000_000000_000063_000000.tif
Threshold mean:  74.36602953414778
Threshold best:  87
Accuracy of mask:  0.6
Bad segmentation mask
Possible number of rhabdomeres:  3
Total number of pixels in the mask:  23205
Total Time Taken:  5.459713697433472  seconds
Number of good masks so far:  0
Number of bad masks so far:  22
Total files processed so far:  22
---------------------------------------------------------
Image Name: 000000_000000_000066_000000.tif
Threshold mean:  72.31582174021683
Threshold best:  84
Accuracy of mask:  0.4
Bad segmentation mask
Possible number of rhabdomeres:  2
T

Threshold mean:  58.26901399346935
Threshold best:  68
Accuracy of mask:  0.3333333333333333
Bad segmentation mask
Possible number of rhabdomeres:  1
Total number of pixels in the mask:  31242
Total Time Taken:  5.508793354034424  seconds
Number of good masks so far:  1
Number of bad masks so far:  39
Total files processed so far:  40
---------------------------------------------------------
Image Name: 000000_000000_000130_000000.tif
Threshold mean:  61.903578877012365
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  5.491065502166748  seconds
Number of good masks so far:  1
Number of bad masks so far:  40
Total files processed so far:  41
---------------------------------------------------------
Image Name: 000000_000000_000131_000000.tif
Threshold mean:  58.29747072680966
Threshold best:  80
Accuracy of mask:  0.16666666666666666
Bad segmentation mask
Possible number of rhabdome

Threshold mean:  48.491001732620134
Threshold best:  67
Accuracy of mask:  0.5555555555555556
Good segmentation mask
Possible number of rhabdomeres:  5
Total number of pixels in the mask:  23644
Total Time Taken:  6.168893814086914  seconds
Number of good masks so far:  7
Number of bad masks so far:  53
Total files processed so far:  60
---------------------------------------------------------
Image Name: 000000_000000_000187_000000.tif
Threshold mean:  46.99113314929027
Threshold best:  65
Accuracy of mask:  0.8181818181818182
Good segmentation mask
Possible number of rhabdomeres:  9
Total number of pixels in the mask:  24214
Total Time Taken:  6.68546199798584  seconds
Number of good masks so far:  8
Number of bad masks so far:  53
Total files processed so far:  61
---------------------------------------------------------
Image Name: 000000_000000_000188_000000.tif
Threshold mean:  46.46668424798455
Threshold best:  61
Accuracy of mask:  0.5
Good segmentation mask
Possible number of 

...

Threshold mean:  44.85852663857597
Threshold best:  61
Accuracy of mask:  0.625
Good segmentation mask
Possible number of rhabdomeres:  5
Total number of pixels in the mask:  21170
Total Time Taken:  3.1037135124206543  seconds
Number of good masks so far:  158
Number of bad masks so far:  54
Total files processed so far:  212
---------------------------------------------------------
Image Name: 000000_000000_000565_000000.tif
Threshold mean:  43.16679376560254
Threshold best:  57
Accuracy of mask:  0.4444444444444444
Good segmentation mask
Possible number of rhabdomeres:  4
Total number of pixels in the mask:  24520
Total Time Taken:  3.101743698120117  seconds
Number of good masks so far:  159
Number of bad masks so far:  54
Total files processed so far:  213
---------------------------------------------------------
Image Name: 000000_000000_000566_000000.tif
Threshold mean:  43.21195163744272
Threshold best:  41
Accuracy of mask:  0.6666666666666666
Good segmentation mask
Possible n

Threshold mean:  98.19824480049778
Threshold best:  126
Accuracy of mask:  0.3333333333333333
Bad segmentation mask
Possible number of rhabdomeres:  2
Total number of pixels in the mask:  23956
Total Time Taken:  2.9671669006347656  seconds
Number of good masks so far:  162
Number of bad masks so far:  69
Total files processed so far:  231
---------------------------------------------------------
Image Name: 000000_000000_000628_000000.tif
Threshold mean:  98.91606181619804
Threshold best:  129
Accuracy of mask:  0.3333333333333333
Bad segmentation mask
Possible number of rhabdomeres:  2
Total number of pixels in the mask:  22698
Total Time Taken:  2.946678638458252  seconds
Number of good masks so far:  162
Number of bad masks so far:  70
Total files processed so far:  232
---------------------------------------------------------
Image Name: 000000_000000_000629_000000.tif
Threshold mean:  98.58243962172622
Threshold best:  108
Accuracy of mask:  0.2727272727272727
Bad segmentation ma

Threshold mean:  7.052124638260504e-06
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  1.233166217803955  seconds
Number of good masks so far:  162
Number of bad masks so far:  89
Total files processed so far:  251
---------------------------------------------------------
Image Name: 000000_000000_000691_000000.tif
Threshold mean:  0.015274412029392118
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  5596
Total Time Taken:  1.386481523513794  seconds
Number of good masks so far:  162
Number of bad masks so far:  90
Total files processed so far:  252
---------------------------------------------------------
Image Name: 000000_000000_000697_000000.tif
Threshold mean:  0.19931381264012377
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of 

Threshold mean:  11.131000335399921
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  1.6729152202606201  seconds
Number of good masks so far:  162
Number of bad masks so far:  109
Total files processed so far:  271
---------------------------------------------------------
Image Name: 000000_000000_000763_000000.tif
Threshold mean:  11.981757561365763
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  1.693479299545288  seconds
Number of good masks so far:  162
Number of bad masks so far:  110
Total files processed so far:  272
---------------------------------------------------------
Image Name: 000000_000000_000768_000000.tif
Threshold mean:  13.462342057909286
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels

Threshold mean:  11.482355238317135
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  1.677600622177124  seconds
Number of good masks so far:  162
Number of bad masks so far:  129
Total files processed so far:  291
---------------------------------------------------------
Image Name: 000000_000000_000830_000000.tif
Threshold mean:  3.8038474832262317
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  1.4134714603424072  seconds
Number of good masks so far:  162
Number of bad masks so far:  130
Total files processed so far:  292
---------------------------------------------------------
Image Name: 000000_000000_000835_000000.tif
Threshold mean:  3.4450619273133336
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels